# Robust HMF on *BOSS* spectra of hot stars...
...to find evidence of H-alpha emission.

## Authors:
- **David W. Hogg** (NYU) (MPIA) (Flatiron)
- **Hans-Walter Rix** (MPIA)

## To-do items:
- Vet results and deliver to HWR's people.
- Make some method (perhaps in `rhmf.py`) to save and restore a Robust HMF model.

## Bugs:
- Probably RHMF is the wrong tool for this job?

In [ ]:
# Cell 1: Import required libraries
import numpy as np
import pandas as pd
from astropy.io import fits
import requests
from requests.auth import HTTPBasicAuth
import os
import concurrent.futures
import matplotlib.pyplot as plt
import rhmf

In [ ]:
# data choices
bosstag = 'v6_2_1'
cachedir = f'./boss_{bosstag}_star_cache'
os.makedirs(cachedir, exist_ok=True)

In [ ]:
# model choices
rank, nsigma = 24, 3.5

In [ ]:
# Define download functions
user, password = None, None

def download_one_file_from_df(args):
    """Download a single file from SDSS."""
    url, filename, user, password, cachedir = args
    filepath = os.path.join(cachedir, filename)
    
    # Skip if already downloaded
    if os.path.exists(filepath):
        # print(f"File {filename} already exists, skipping")
        return True
        
    try:
        with requests.Session() as session:
            response = session.get(url, auth=HTTPBasicAuth(user, password), timeout=30)
            response.raise_for_status()
            with open(filepath, 'wb') as f:
                f.write(response.content)
        if np.random.uniform() < 0.1:
            print(f"Random example: File downloaded: {filename}")
        return True
    except Exception as e:
        print(f"Failed to download {filename}: {e}")
        return False

def download_files_from_df(df, user, password, dest_folder, boss_tag='v6_2_1', coadd_version='daily', max_workers=8):
    """Download multiple files from SDSS based on dataframe."""
    os.makedirs(dest_folder, exist_ok=True)
    args_list = []

    for idx, row in df.iterrows():
        spec_file = row['SPEC_FILE']
        fieldid = f"{row['FIELD']:06d}"
        mjd = str(row['MJD'])
        fieldidXXX = fieldid[:-3] + 'XXX'
        url = (
            f"https://data.sdss5.org/sas/sdsswork/bhm/boss/spectro/redux/"
            f"{boss_tag}/spectra/{coadd_version}/lite/{fieldidXXX}/{fieldid}/{mjd}/{spec_file}"
        )
        args_list.append((url, spec_file, user, password, dest_folder))

    print(f"Starting attempts to download {len(args_list)} files")
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(download_one_file_from_df, args_list))
    print(f"Number successful: {sum(results)} files")
    return results

In [ ]:
# Download and examine the summary file
spallname = f'spAll-lite-{bosstag}.fits'
summaryurl = f'https://data.sdss5.org/sas/sdsswork/bhm/boss/spectro/redux/{bosstag}/summary/daily/{spallname}.gz'
summaryfile = cachedir + '/' + spallname + '.gz'
summaryfile_uncompressed = cachedir + '/' + spallname

if not os.path.exists(summaryfile_uncompressed):
    if not os.path.exists(summaryfile):
        print(f"Downloading summary file from {summaryurl}")
        response = requests.get(summaryurl, auth=HTTPBasicAuth(user, password))
        with open(summary_file, 'wb') as f:
            f.write(response.content)
        print(f"Summary file {summaryfile} downloaded")
    
    # Decompress
    os.system(f'gunzip -v {summaryfile}')
    print(f"Summary file {summaryfile} decompressed")
else:
    print(f"Summary file {summaryfile_uncompressed} already exists")

In [ ]:
# Load summary data
with fits.open(summaryfile_uncompressed) as hdul:
    data = hdul[1].data

if False:
    print("="*70)
    print("ALL AVAILABLE COLUMNS IN SUMMARY FILE")
    print("="*70)
    columns = data.columns.names
    for i, col in enumerate(columns):
        print(f"{i+1:3d}. {col}")

print(f"rows: {len(data)}; columns: {len(data.columns.names)}")

In [ ]:
# Select a sample of spectra to download
# Let's look for high-SNR spectra with BP_MAG - RP_MAG < 0.5 to examine
df = pd.DataFrame({col: data[col].byteswap().newbyteorder() for col in ['SPEC_FILE', 'FIELD', 'MJD', 'SN_MEDIAN_ALL', 'BP_MAG', 'RP_MAG', 'PROGRAMNAME']})

# Filter for high SNR and BP_MAG - RP_MAG < 0.5 spectra
high_snr_df = df[(df['SN_MEDIAN_ALL'] > 40) & (df['PROGRAMNAME'] == 'mwm_ob') ] #.head(50)  # Just 50 spectra for testing
print(f"Selected {len(high_snr_df)} high-SNR and BP-RP < 0.5 spectra for header examination")

In [ ]:
# Download the sample spectra
_ = download_files_from_df(high_snr_df, user, password, cachedir, boss_tag=bosstag, coadd_version='daily', max_workers=4)

In [ ]:
# Examine the headers of downloaded spectra
if False:
    print("\n" + "="*70)
    print("EXAMINING SPECTRA HEADERS")
    print("="*70)
    
    spec_files = [f for f in os.listdir(cachedir) if f.endswith('.fits')][:3]  # Examine first 3
    
    for i, spec_file in enumerate(spec_files):
        print(f"{'='*60}")
        print(f"SPECTRUM {i+1}: {spec_file}")
        print(f"{'='*60}")
        
        filepath = os.path.join(cachedir, spec_file)
        with fits.open(filepath) as hdul:
            print(f"\nNumber of HDUs: {len(hdul)}")
            
            for j, hdu in enumerate(hdul):
                print(f"\n--- HDU {j}: {type(hdu).__name__} ---")
                
                # Print header info
                if hdu.header:
                    print(f"Header cards: {len(hdu.header)} cards")
                    print("\nFirst 20 header keywords:")
                    for k, (key, value) in enumerate(list(hdu.header.items())[:20]):
                        print(f"  {key:8s} = {str(value)[:60]}")
                    if len(hdu.header) > 20:
                        print(f"  ... and {len(hdu.header)-20} more header cards")
                
                # If it's a table, show columns
                if hasattr(hdu, 'columns'):
                    print(f"\nTable columns: {len(hdu.columns)}")
                    for col in hdu.columns:
                        print(f"  - {col.name:20s} ({col.format})")
                
                # If it's image data, show shape
                if hasattr(hdu, 'data') and hdu.data is not None:
                    if isinstance(hdu, fits.PrimaryHDU) or isinstance(hdu, fits.ImageHDU):
                        print(f"\nImage data shape: {hdu.data.shape}")
                    elif isinstance(hdu, fits.BinTableHDU):
                        print(f"\nTable rows: {len(hdu.data)}")

In [ ]:
def plot_one_spectrum(wavelength, flux, ivar, name, prefix, synth=None):
    f = plt.figure(figsize=(12, 4))
    plt.axhline(0., lw=0.5, color='k', alpha=0.45)
    plt.step(wavelength, flux,
             where='mid', color='k', lw=0.5, alpha=0.90)
    tiny = 0.01 / np.median(flux) ** 2
    flhi = flux + 1. / np.sqrt(ivar + tiny)
    fllo = flux - 1. / np.sqrt(ivar + tiny)
    plt.fill_between(wavelength, fllo, flhi,
                     step='mid', color='k', alpha=0.23)
    if synth is not None:
        plt.step(wavelength, flux - synth,
                 where='mid', color='k', lw=0.5, alpha=0.90)
        plt.step(wavelength, synth,
                 where='mid', color='r', lw=0.5, alpha=0.90)
        plt.step(wavelength, np.zeros_like(flux),
                 where='mid', color='r', lw=0.5, alpha=0.90)
    plt.xticks([2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000])
    plt.xlim(np.min(wavelength), np.max(wavelength))
    foo = np.nanmedian(flux)
    plt.ylim(-0.5 * foo, 2.5 * foo)
    plt.semilogx()
    plt.xlabel('wavelength')
    plt.ylabel('flux')
    plt.title(name)
    
    # Save plot
    plot_filename = plot_folder + '/' + prefix + name + '.png'
    plt.savefig(plot_filename)
    plt.close(f)
    print(f"  Plot saved: {plot_filename}")

In [ ]:
# Create subdirectory for plots
plot_folder = cachedir + '/plots'
os.makedirs(plot_folder, exist_ok=True)

# Get list of downloaded FITS files (up to Nmax)
Nmax = 10
spec_files = [f for f in os.listdir(cachedir) if f.endswith('.fits')][:Nmax]
print(f"Found {len(spec_files)} spectra to plot (limited to {Nmax})")

for i, spec_file in enumerate(spec_files):
    filepath = cachedir + "/" + spec_file

    with fits.open(filepath) as hdul:
        if len(hdul) > 1 and hasattr(hdul[1], 'data'):
            spec_data = hdul[1].data
            loglam = spec_data['LOGLAM']
            flux = spec_data['FLUX']
            ivar = spec_data['IVAR']
            wavelength = 10**loglam
            plot_one_spectrum(wavelength, flux, ivar, spec_file, "")

In [ ]:
# make a data block
print("assembling file names for reading")
filenames = np.array([f for f in os.listdir(cachedir) if f.endswith('.fits') and f.startswith('spec-')])
wavelength = None
N = len(filenames)
print(f"reading {N} files...")
for i, fn in enumerate(filenames):
    filepath = cachedir + '/' + fn
    
    try:
        with fits.open(filepath) as hdul:
            if len(hdul) > 1 and hasattr(hdul[1], 'data'):
                spec_data = hdul[1].data
                loglam = spec_data['LOGLAM']
                fl = spec_data['FLUX']
                iv = spec_data['IVAR']
                wa = 10**loglam
                if wavelength is None:
                    wavelength = wa
                    M = len(wavelength)
                    flux = np.ones((N, M))
                    ivar = np.zeros_like(flux)
                if np.allclose(wa, wavelength):
                    flux[i] = fl / np.median(fl)
                    ivar[i] = iv * np.median(fl) ** 2
                else:
                    print(f"  Dropped {filepath}: bad wavelength grid")

    except Exception as e:
        print(f"  Dropped {filepath}: {e}")

print("data blocks:", flux.shape, ivar.shape, np.prod(flux.shape))
print("bad pixels:", np.sum(~ np.isfinite(flux)), np.sum(~ np.isfinite(ivar)),
      np.sum(ivar <= 0.) / np.prod(flux.shape))

In [ ]:
# trim data
good = (wavelength > 3700.) # magic
wavelength = wavelength[good]
flux = flux[:, good]
ivar = ivar[:, good]

In [ ]:
# floor and ceil the ivars ## HACK
maxivar = 1.e4 / flux ** 2 # magic -- nothing is known to better than 1 percent
ivar = np.clip(ivar, 0., maxivar)
maxivar = 1.e4 / np.median(flux, axis=1) ** 2 # magic -- nothing is known to better than 1 percent on average
minivar = 1.e-2 / np.median(flux, axis=1) ** 2 # magic -- there is some information even at useless pixels
ivar = np.clip(ivar, minivar[:, None], maxivar[:, None])
print(np.min(ivar), np.max(ivar))

In [ ]:
# make two disjoint training sets
N, M = flux.shape
rng = np.random.default_rng(17)
foo = np.random.uniform(size=N)
A = foo < np.median(foo)
B = np.logical_not(A)
print(np.sum(A), np.sum(B), ~np.any(np.logical_and(A, B)))

In [ ]:
Aidx = np.arange(N)[A]
Bidx = np.arange(N)[B]
print(len(Aidx), len(Bidx), np.all(A[Aidx]), np.all(B[Bidx]))

In [ ]:
def plot_G(model, title):
    plt.figure(figsize=(12,8))
    for k, g in enumerate(model.G):
        plt.step(wavelength, 10. * g + k,
                 where='mid', lw=0.5, alpha=0.90)
    plt.xlabel("wavelength")
    plt.xlim(np.min(wavelength), np.max(wavelength))
    plt.semilogx()
    plt.ylim(-1., model.K)
    plt.title(title)

In [ ]:
def cross_test(Y, W, models):
    assert len(models) == 2
    synth = np.zeros_like(flux) + np.nan
    print(np.sum(np.isnan(synth)))
    for m in range(2):
        n = (m + 1) % 2
        model, _, _ = models[m]
        _, idx, _ = models[n]
        for i in idx:
            synth[i] = model.test(Y[i], W[i])
        print(np.sum(np.isnan(synth)))    
    return synth

def train_and_test(Y, W, models, maxiter=10):
    for model, idx, label in models:
        print(label)
        model.train(Y[idx], W[idx], maxiter=maxiter)
        plot_G(model, label)
        plt.show()
    synth = cross_test(Y, W, models)
    for i in range(100):
        plot_one_spectrum(wavelength, Y[i], W[i], filenames[i], "", synth=synth[i])
    return synth

In [ ]:
# start models
models = [(rhmf.RHMF(rank, nsigma), Aidx, "model A"),
          (rhmf.RHMF(rank, nsigma), Bidx, "model B")]
synth = train_and_test(flux, ivar, models, maxiter=30)

In [ ]:
# train more
synth = train_and_test(flux, ivar, models, maxiter=100)

In [ ]:
# train even more
synth = train_and_test(flux, ivar, models, maxiter=300)

In [ ]:
# now re-run the test step but with the H-alpha region censored.
halpha = 6564.6
delta = 6. # magic
near_halpha = (wavelength > (halpha - delta)) & (wavelength < (halpha + delta))
print(np.sum(near_halpha))
ivar_halpha = 1. * ivar # copy
ivar_halpha[:, near_halpha] = 0.
synth_halpha = cross_test(flux, ivar_halpha, models)

In [ ]:
# now order the results by the significance of the H-alpha region deviation (chi)
resid = flux - synth_halpha
chi_halpha = (resid * np.sqrt(ivar))[:, near_halpha]
print(chi_halpha.shape)
chi2_halpha = np.sum(chi_halpha ** 2, axis=1)
interesting = np.argsort(-chi2_halpha)
_ = plt.hist(np.log10(chi2_halpha), bins=100)
plt.xlabel("log10(chi-squared)")
plt.semilogy()

In [ ]:
# plot the best candidates for delivery to our people
for i in interesting[:200]:
    plot_one_spectrum(wavelength, flux[i], ivar[i], filenames[i], "halpha-", synth=synth_halpha[i])